# Non-discrete Optimization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gaussian_process.GPfunctions as gp
from gaussian_process import GaussianProcess
from gaussian_process.kernels import CubicKernel, SquaredExponentialKernel
from acquisition import (
    LowerConfidenceBoundVariance,
    GP_LCB_Variance,
)
from acquisition.optimization import (
    ScipyAcquisitionOptimizer,
    GradientBinarySearchAcquisitionOptimizer,
    DIRECTAcquisitionOptimizer,
)

In [ ]:
# Objective function

objectiveFunction = lambda x: -x * np.sin(x)
objectiveFunctionDerivative = lambda x: -x * np.cos(x) - np.sin(x)

X = np.linspace(start=-3.0, stop=3.0, num=1_000)
y = objectiveFunction(X)

In [ ]:
max_iter = 10
acquisitionOptimizer = DIRECTAcquisitionOptimizer()

## Minimization

In [ ]:
X_sample = np.array([-3.0, 3.0])
y_sample = objectiveFunction(X_sample)

x_best = X_sample[np.argmin(y_sample)]
y_best = np.min(y_sample)

for t in range(max_iter):
    kernel = SquaredExponentialKernel(l=np.abs(X[0] - X[-1]) / (len(X_sample) - 1))
    GP_posterior = GaussianProcess(kernel=kernel, x_known=X_sample, f_known=y_sample)

    mean, variance = GP_posterior(X)

    fig, (ax1) = plt.subplots(1, 1, sharey=True)
    gp.plot_objective(ax1, X, y, X_sample, y_sample)
    gp.plot_gp(ax1, X, mean, variance)

    acquisitionFunction = GP_LCB_Variance(GP_posterior, t, 1, nu=10)

    ax1.plot(X, [acquisitionFunction(x) for x in X], label="Acquisition")
    ax1.plot(
        X, [acquisitionFunction.derivative(x) for x in X], label="Acquisition Gradient"
    )

    gp.plot_label(ax1, f"{len(X_sample)} data points")
    plt.show()

    maxAquisition = acquisitionOptimizer.maximize(
        acquisitionFunction, -3.0, 3.0, X_sample
    )

    x_new = maxAquisition
    y_new = objectiveFunction(x_new)
    if x_new in X_sample:
        print("Terminated due to insufficient acquisition function")
    else:
        X_sample = np.append(X_sample, x_new)
        y_sample = np.append(y_sample, y_new)
        if y_new < y_best:
            x_best, y_best = x_new, y_new
    print(f"best: x={x_best}, y={y_best}")

print(f"best: ({x_best},{y_best})")

## Minimization with gradient

In [ ]:
X_sample = np.array([-3.0, 3.0])
y_sample = objectiveFunction(X_sample)
g_sample = objectiveFunctionDerivative(X_sample)

x_best = X_sample[np.argmin(y_sample)]
y_best = np.min(y_sample)

for t in range(max_iter):
    kernel = SquaredExponentialKernel(
        l=np.abs(X[0] - X[-1]) / (2 * (len(X_sample) - 1))
    )
    GP_posterior = GaussianProcess(
        kernel=kernel, x_known=X_sample, f_known=y_sample, g_known=g_sample
    )

    mean, variance = GP_posterior(X)

    fig, (ax1) = plt.subplots(1, 1, sharey=True)
    gp.plot_objective(ax1, X, y, X_sample, y_sample)
    gp.plot_gp(ax1, X, mean, variance)

    acquisitionFunction = GP_LCB_Variance(GP_posterior, t, 1, nu=10)

    ax1.plot(X, [acquisitionFunction(x) for x in X], label="Acquisition")
    ax1.plot(
        X, [acquisitionFunction.derivative(x) for x in X], label="Acquisition Gradient"
    )

    gp.plot_label(ax1, f"{len(X_sample)} data points")
    plt.show()

    maxAquisition = acquisitionOptimizer.maximize(
        acquisitionFunction, -3.0, 3.0, X_sample
    )

    x_new = maxAquisition
    y_new = objectiveFunction(x_new)
    g_new = objectiveFunctionDerivative(x_new)
    if x_new in X_sample:
        print("Terminated due to insufficient acquisition function")
    else:
        X_sample = np.append(X_sample, x_new)
        y_sample = np.append(y_sample, y_new)
        g_sample = np.append(g_sample, g_new)
        if y_new < y_best:
            x_best, y_best = x_new, y_new
    print(f"best: x={x_best}, y={y_best}")

print(f"best: ({x_best},{y_best})")